In [2]:
# from libs.alibaba.visitor import Visitor
from libs.alibaba.alibaba import Alibaba
from libs.json import JSON
from openpyxl import load_workbook
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import os
import traceback
import types
import pendulum

In [3]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']

. storage markets.json
.\storage\markets.json was deserialized!


In [4]:
class Product_Ranking:
    api = 'https://hz-productposting.alibaba.com/product/ranksearch/rankSearch.htm'
    browser = None
    
    def __init__(self, market, headless=True, browser=None):
        self.market = market
        self.lid = market['lid']
        self.lpwd = market['lpwd']

        self.alibaba = None
        self.browser = browser
        self.headless = headless
        
    def login(self):
        self.alibaba = Alibaba(self.lid, self.lpwd, headless=self.headless, browser=self.browser)
        self.alibaba.login()
        self.browser = self.alibaba.browser

In [5]:
pr = Product_Ranking(market, headless=False)

In [6]:
pr.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [8]:
pr.browser.get(pr.api)

In [51]:
def find_product_rank(self, kw, product_id):
    result = None
    input = self.browser.find_element_by_css_selector('#queryString')
    input.send_keys(Keys.CONTROL, 'a')
    input.send_keys(kw)
    input.send_keys(Keys.RETURN)
    
    css_table = '#rank-searech-table'
    table = WebDriverWait(self.browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_table))
    )
    for tr in table.find_elements_by_css_selector('tbody tr'):
        a = tr.find_element_by_css_selector('td.products a')
        if product_id in a.get_attribute('href'):
            a = tr.find_element_by_css_selector('td.ranking a')
            url = a.get_attribute('href')
            text = a.text
            page = re.search('约第(\d+)页，第(\d+)位', text).group(1)
            pos = re.search('约第(\d+)页，第(\d+)位', text).group(2)
            result = {'keyword':kw, 'page':page, 'position':pos}
            break
    return result
pr.find_product_rank = types.MethodType(find_product_rank, pr)

In [102]:
file = 'eyelash_product_ranking.xlsx'
wb = load_workbook(file)
sheet = wb.worksheets[0]
pid = str(sheet.cell(row=1,column=2).value)
for i in range(6, 9999):
    cell = sheet.cell(row=i,column=1)
    kw = cell.value
    if kw is None:
        break
    print(kw, end='\t')
    ranking = pr.find_product_rank(kw, pid)
    if ranking:
        result = ranking['page'] + '-' + ranking['position']
    else:
        result = 'X'
    cell = sheet.cell(row=i,column=2)
    cell.value = result
    print(result)
wb.save(file)

eyelash	X
lash	X
false eyelash	X
false lash	X
wholesale eyelash	X
wholesale lash	X
eyelash vendor	X
lash vendor	X
eyelash distributor	X
lash distributor	X
private label eyelash	X
private label lash	3-1
eyelash private label	X
lash private label	X
own brand eyelash	X
own brand lash	X
eyelash extension	X
silk eyelash extension	X
korean eyelash extension	X
premium eyelash extension	X
lash extension	X
individual eyelash	X
individual lash	2-19
individual eyelash extension	X
individual lash extension	X
eyelash extension private label	X
lash extension private label	X
private label eyelash extension	X
private label lash extension	X
synthetic mink eyelash extension	X
korean eyelash	X
